# Tutorial de Neural Style Transfer com o Keras
To run this, please use the GPU hardware acceleration in Colab

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import vgg19

In [ ]:
base_image_path = "/content/santos.jpg"
style_reference_image_path = keras.utils.get_file(
    "starry_night.jpg", "https://i.imgur.com/9ooB60I.jpg"
)

result_prefix = "dog_van_gogh"

In [ ]:
#Pesos de cada tipo de perda

total_variation_weight= 1e-6  # esse tipo de perda se refere a coerência visual
# da imagem final, impondo aos pixels dela continuidade espacial local, ou seja,
# quanto menor esse erro, mas a imagem faz sentido e evita um pixel ser de outra
# cor de uma forma mto sem sentido.

style_weight = 1e-6  # essa perda tem a ver com deep learning e é tipo o
# o resultado da soma das distâncias em L2 das matrizes de Gram 
# que representam as imagens e q foram extraídas de cada layer da Convnet. A 
# ideia por traz disso é capturar a disposição de cor e textura das imagens à 
# diferença espacial, definido a uma escala grande que por sua vez tem a ver com
# cada layer da CNN. 

# Como q isso funciona é assim: 
# Uma CNN pega cada informações da imagem e forma matrizes com elas, 
# tipo uma matriz q tem informações sobre as linhas, sobre as colunas, enfim
# caracteristicas da pintura. O estilo não pode ser computado dessa forma porque
# ele é meio q uma combinação desses elementos, ent o q se faz é transformar
# essas matrizes numa linha e multiplicar pela transposta, o q seria uma Matriz
# de Gram

content_weight = 2.5e-8 
# Essa perda tem a ver com deixar a imagem resultante mais próxima da original,
# tbm faz a soma das distancias em L2 entre a img resultante e a original. 

In [ ]:
width, height = keras.preprocessing.image.load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

In [ ]:
from IPython.display import Image, display

display(Image(base_image_path))
display(Image(style_reference_image_path))

# Fazer as funções de transformar a imagem em tensor e vice-versa

In [ ]:
def preprocess_image(image_path):
    # Util function to open, resize and format pictures into appropriate tensors
    img = keras.preprocessing.image.load_img(
        image_path, target_size=(img_nrows, img_ncols)
    )
    img = keras.preprocessing.image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return tf.convert_to_tensor(img)

def deprocess_image(x):
  x = x.reshape((img_nrows, img_ncols, 3))

  # remove zero center
  x[:, :, 0] += 103.939
  x[:, :, 1] += 116.779
  x[:, :, 2] += 123.68

  # de BGR p/ RGB
  x = x[:, :, ::-1]
  x = np.clip(x, 0, 255).astype("uint8")
  return x

# Fazer as funções p/ computar a matriz de gram e as perdas

In [ ]:
def gram_matrix(x):  # Função pra criar a Matriz de Gram
  x = tf.transpose(x, (2,0,1))
  features = tf.reshape(x, (tf.shape(x)[0], -1))
  gram = tf.matmul(features, tf.transpose(features))
  return gram

def style_loss(style, combination): # computa a perda de "estilo"
  S = gram_matrix(style)
  C = gram_matrix(combination)
  channels = 3
  size = img_nrows * img_ncols
  return tf.reduce_sum(tf.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

def content_loss(base, combination): # computa a perda no conteudo
  return tf.reduce_sum(tf.square(combination - base))

def total_variation_loss(x): # computa a perda de variação total, ou seja, coerencia visual
    a = tf.square(
        x[:, : img_nrows - 1, : img_ncols - 1, :] - x[:, 1:, : img_ncols - 1, :]
    )
    b = tf.square(
        x[:, : img_nrows - 1, : img_ncols - 1, :] - x[:, : img_nrows - 1, 1:, :]
    )
    return tf.reduce_sum(tf.pow(a + b, 1.25))

In [ ]:
# Usando um modelo pré treinado de VGG19
model = vgg19.VGG19(weights="imagenet", include_top = False)

outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

feature_extractor = keras.Model(inputs = model.inputs, outputs = outputs_dict)

80150528/80134624 [==============================] - 0s 0us/step


In [ ]:
style_layer_names = [
    "block1_conv1",
    "block2_conv1",
    "block3_conv1",
    "block4_conv1",
    "block5_conv1",
]

content_layer_name = "block5_conv2"

def compute_loss(combination_image, base_image, style_reference_image):
  # Essa função computa a perda total
    input_tensor = tf.concat(
        [base_image, style_reference_image, combination_image], axis=0
    )
    features = feature_extractor(input_tensor)

    # Initialize the loss
    loss = tf.zeros(shape=())

    # Add content loss
    layer_features = features[content_layer_name]
    base_image_features = layer_features[0, :, :, :]
    combination_features = layer_features[2, :, :, :]
    loss = loss + content_weight * content_loss(
        base_image_features, combination_features
    )
    # Add style loss
    for layer_name in style_layer_names:
        layer_features = features[layer_name]
        style_reference_features = layer_features[1, :, :, :]
        combination_features = layer_features[2, :, :, :]
        sl = style_loss(style_reference_features, combination_features)
        loss += (style_weight / len(style_layer_names)) * sl

    # Add total variation loss
    loss += total_variation_weight * total_variation_loss(combination_image)
    return loss

In [ ]:
@tf.function
def compute_loss_and_grads(combination_image, base_image, style_reference_image):
  # essa 
    print("cheguei aq")
    with tf.GradientTape() as tape:
        loss = compute_loss(combination_image, base_image, style_reference_image)
    grads = tape.gradient(loss, combination_image)
    return loss, grads

In [ ]:
optimizer = keras.optimizers.SGD(
    keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=100.0, decay_steps=100, decay_rate=0.96
    )
)

base_image = preprocess_image(base_image_path)
style_reference_image = preprocess_image(style_reference_image_path)
combination_image = tf.Variable(preprocess_image(base_image_path))

iterations = 4000
for i in range(1, iterations + 1):
    loss, grads = compute_loss_and_grads(
        combination_image, base_image, style_reference_image
    )
    optimizer.apply_gradients([(grads, combination_image)])
    if i % 100 == 0:
        print("Iteration %d: loss=%.2f" % (i, loss))
        img = deprocess_image(combination_image.numpy())
        fname = result_prefix + "_at_iteration_%d.png" % i
        keras.preprocessing.image.save_img(fname, img)

In [ ]:
display(Image(result_prefix + "_at_iteration_4000.png"))